In [ ]:
# Example code for building and deploying a machine learning model on a cloud platform
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
import pickle
import os
import boto3

In [ ]:
# Load data
df = pd.read_csv('data.csv')

In [ ]:
df.head()

In [ ]:
# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(df.drop('target', axis=1), df['target'], test_size=0.3)

In [ ]:
# Train model
clf = LogisticRegression()
clf.fit(X_train, y_train)

In [ ]:
# Save model to disk
filename = 'model.pkl'
pickle.dump(clf, open(filename, 'wb'))

In [ ]:
# Set AWS credentials
ACCESS_KEY = 'your-access-key'
SECRET_KEY = 'your-secret-key'

In [ ]:
# Upload model to cloud platform
s3 = boto3.resource('s3', aws_access_key_id=ACCESS_KEY, aws_secret_access_key=SECRET_KEY)
bucket_name = 'my-bucket'
bucket = s3.Bucket(bucket_name)
bucket.upload_file(filename, filename)

In [ ]:
# Deploy model on cloud platform
from sagemaker import Model
from sagemaker.predictor import RealTimePredictor

model_data = 's3://{}/{}'.format(bucket_name, filename)
model = Model(model_data=model_data, role='SageMakerRole', image='my-image')
predictor = model.deploy(initial_instance_count=1, instance_type='ml.t2.medium')

In [ ]:
# Use model to make predictions
data = np.array([[1, 2, 3, 4], [5, 6, 7, 8]])
predictions = predictor.predict(data)
print(predictions)